## なろうデータ取得

In [1]:
import requests
import pandas as pd
import json
import time as tm
import datetime
import gzip
from tqdm.notebook import tqdm

2020年のデータを取得

In [2]:
input_dt = datetime.datetime.strptime("2020/1/1","%Y/%m/%d")
output_dt = datetime.datetime.strptime("2021/1/1","%Y/%m/%d")
input_dt.timestamp

<function datetime.timestamp>

In [3]:
#リクエストの秒数間隔(1以上を推奨)
interval = 2
### なろう小説API・なろう１８禁小説API を設定 ####
api_url="https://api.syosetu.com/novelapi/api/"    
    
#全作品情報の取得
def get_all_novel_info(input_dt,output_dt):
    lastup = output_dt.timestamp()    
       
    df = pd.DataFrame()
    payload = {'out': 'json','gzip':5,'of':'n','lim':1,'lastup':f"{input_dt.timestamp()}-{lastup}"}
    res = requests.get(api_url, params=payload).content
    r =  gzip.decompress(res).decode("utf-8") 
    allcount = json.loads(r)[0]["allcount"]
    print('対象作品数  ',allcount);
    
    all_queue_cnt = (allcount // 500) + 10
                     
    for i in tqdm(range(all_queue_cnt)):
        payload = {'out': 'json','gzip':5,'opt':'weekly','lim':500,'lastup':f"{input_dt.timestamp()}-{lastup}"}
        
        # なろうAPIにリクエスト
        cnt=0
        while cnt < 5:
            try:
                res = requests.get(api_url, params=payload, timeout=30).content
                break
            except:
                print("Connection Error")
                cnt = cnt + 1
                tm.sleep(120) #接続エラーの場合、120秒後に再リクエストする
            
        r =  gzip.decompress(res).decode("utf-8")   
    
        # pandasのデータフレームに追加する処理
        df_temp = pd.read_json(r)
        df_temp = df_temp.drop(0)
        df = pd.concat([df, df_temp])
        
        last_general_lastup = df.iloc[-1]["general_lastup"]
        lastup = datetime.datetime.strptime(last_general_lastup, "%Y-%m-%d %H:%M:%S").timestamp()
        lastup = int(lastup)
        #取得間隔を空ける
        tm.sleep(interval)
    return df

In [4]:
narou = get_all_novel_info(input_dt,output_dt)

対象作品数   100239


  0%|          | 0/210 [00:00<?, ?it/s]

In [5]:
narou.head()

,allcount,title,ncode,userid,writer,story,biggenre,genre,gensaku,keyword,...,fav_novel_cnt,impression_cnt,review_cnt,all_point,all_hyoka_cnt,sasie_cnt,kaiwaritu,novelupdated_at,updated_at,weekly_unique
1,NaN,これからの運命,N9310GR,2028788.0,きんたつ。,占い好きなやつが新年初日で何かに気づかされるお話。,1.0,102.0,,日常 青春 おみくじ 初詣 三が日 ダイビングパーツ きんたつ。,...,0.0,0.0,0.0,0.0,0.0,0.0,66.0,2021-01-01 09:00:09,2021-01-05 08:35:15,0.0
2,NaN,猫と女の子の探し物,N9423GR,1309365.0,雪氷華,元気な気持ちが空から何処かに落っこちてしまいました。\n猫と小さな女の子がそれを探して見付け...,99.0,9901.0,,冬童話2021,...,0.0,3.0,0.0,36.0,4.0,0.0,5.0,2021-01-01 09:08:11,2022-03-31 19:14:14,0.0
3,NaN,私の走馬灯,N9422GR,2083111.0,Navel,こんなこともあるかも...,3.0,302.0,,日常 青春 自殺 学生,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-01-01 08:47:24,2021-01-05 08:35:15,0.0
4,NaN,アンデルセンの初恋,N8964GR,368733.0,椎名,ある魔法使いの、物語の裏側の恋のお話。\n\nNL/悲恋/童話風/外見ショタ/恋愛＜友情/片...,99.0,9901.0,,悲恋 童話風 恋愛＜友情 外見ショタ魔法使い 両片想い,...,3.0,1.0,0.0,10.0,1.0,0.0,46.0,2021-01-01 08:38:09,2022-09-09 06:31:46,0.0
5,NaN,ためしとて,N8761GR,1183606.0,moes,新年会の後、面倒な同僚に付きまとわれて、面倒な噂話に巻き込まれて、それでもうれしいこともあった。,2.0,202.0,,伝奇 日常 ほのぼの 現代 ハッピーエンド 新年 神様,...,0.0,0.0,0.0,0.0,0.0,0.0,54.0,2022-02-01 22:23:34,2022-02-01 22:26:05,0.0


In [6]:
# 保存
narou.to_pickle("./narou.pkl")

## T5の作成

In [7]:
!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import re
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

from sklearn.model_selection import train_test_split,KFold
import random
import logging


# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [9]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 300

## データの読み込み

In [10]:
narou = pd.read_pickle("./narou.pkl")
narou = narou.dropna(subset=["title","story"])
narou["title"] = narou["title"].astype(str)
narou["story"] = narou["story"].astype(str)

In [11]:
tokenizer = T5Tokenizer.from_pretrained("sonoisa/t5-base-japanese")
pad = 0

## データセットの作成

In [12]:
# 上記のinput_dataの作成方法をもとにdataset化
class NarouDataset(torch.utils.data.Dataset):
    def __init__(self, input_texts,input_max_len, target_texts = None,target_max_len=None):
        self.train_flg = 0 if target_texts is None else 1
        temp_dicts = tokenizer.batch_encode_plus(
                    input_texts, max_length=input_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )
        
        self.input_ids = temp_dicts["input_ids"]
        self.input_masks = temp_dicts["attention_mask"]
        if self.train_flg:
            temp_dicts = tokenizer.batch_encode_plus(
                        target_texts, max_length=input_max_len, truncation=True, 
                        padding="max_length", return_tensors="pt"
                    )

            self.target_ids = temp_dicts["input_ids"]
            self.target_masks = temp_dicts["attention_mask"]
        

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if self.train_flg:
                return (self.input_ids[idx].squeeze(), self.input_masks[idx].squeeze(),
                        self.target_ids[idx].squeeze(), self.target_masks[idx].squeeze() )
        else:
                return self.input_ids[idx].squeeze(), self.input_masks[idx].squeeze()

In [13]:
X_train, X_test, y_train, y_test = train_test_split(narou["story"],narou["title"],test_size=0.1,random_state = 1)
X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train,test_size=0.2,random_state = 1)

In [14]:
def collate_fn(batch):
    input_ids,input_masks,target_ids,target_masks = list(zip(*batch))
    input_max_batch_len = max(map(sum,input_masks)).item()
    target_max_batch_len = max(map(sum,target_masks)).item()
    new_input_ids = torch.Tensor([list(i[:input_max_batch_len]) for i in input_ids])
    new_input_masks = torch.Tensor([list(i[:input_max_batch_len]) for i in input_masks])
    new_target_ids = torch.Tensor([list(i[:target_max_batch_len]) for i in target_ids])
    new_target_masks = torch.Tensor([list(i[:target_max_batch_len]) for i in target_masks])

    return (new_input_ids.type(torch.LongTensor),
            new_input_masks.type(torch.LongTensor),
            new_target_ids.type(torch.LongTensor),
            new_target_masks.type(torch.LongTensor),
           )

In [15]:
# datasetとdataloaderの作成
batch_size=4
train_ds = NarouDataset(input_texts=X_train,input_max_len=256, target_texts = y_train,target_max_len=100)
train_dl = torch.utils.data.DataLoader(
    train_ds, batch_size=batch_size, shuffle=True,collate_fn=collate_fn)  
valid_ds = NarouDataset(input_texts=X_valid,input_max_len=256, target_texts = y_valid,target_max_len=100)
valid_dl = torch.utils.data.DataLoader(
    valid_ds, batch_size=batch_size, shuffle=False,collate_fn=collate_fn)

## モデルの定義

In [16]:
model = T5ForConditionalGeneration.from_pretrained("sonoisa/t5-base-japanese")
cuda = torch.cuda.is_available()
#cuda = False
# 高速化
torch.backends.cudnn.benchmark = True
if cuda:
    model.cuda()
# optimizerの設定
optimizer = optim.Adam(model.parameters(),lr = 5e-5 ,betas=(0.9, 0.999))

# 損失関数の設定
criterion = nn.CrossEntropyLoss()

## 学習

In [17]:
## 時間があれば、epoch数を増やしてみてください。
epochs = 1
best_loss = 9999999
for epoch in range(epochs):
    total_loss = 0
    model.train()
  

    print("epoch {} start!".format(epoch+1))
    logging.info("epoch {} start!".format(epoch+1))
    # train
    for iter_num, (input_ids, input_masks, target_ids, target_masks) in tqdm(enumerate(train_dl),total = len(train_dl)):
        optimizer.zero_grad()
        if cuda:
            input_ids, input_masks, target_ids, target_masks =\
            input_ids.cuda(), input_masks.cuda(), target_ids.cuda(), target_masks.cuda()
        if len(input_ids)>1:
            # forward(順伝搬)
            outputs = model(
                input_ids=input_ids,
                attention_mask=input_masks,
                labels=target_ids)
            loss = outputs.loss

            # backward(逆伝搬)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
    total_loss /= len(train_dl)

    # validation
    val_total_loss = 0
    model.eval()
    for iter_num, (input_ids, input_masks, target_ids, target_masks) in tqdm(enumerate(valid_dl),total = len(valid_dl)):
        optimizer.zero_grad()
        if cuda:
            input_ids, input_masks, target_ids, target_masks =\
            input_ids.cuda(), input_masks.cuda(), target_ids.cuda(), target_masks.cuda()
        with torch.no_grad():
            # forward(順伝搬)
            outputs = model(
                input_ids=input_ids,
                attention_mask=input_masks,
                labels=target_ids)
            loss = outputs.loss
            val_total_loss += loss.item()
    val_total_loss /= len(valid_dl)

    print("epoch{} total loss:{:.4f}, val_total loss:{:.4f}".format(epoch+1,total_loss,val_total_loss))
    logging.info("epoch{} total loss:{:.4f}, val_total loss:{:.4f}".format(epoch+1,total_loss,val_total_loss))
    torch.save(model.state_dict(), f'./T5model_last.hdf5')
    if best_loss > val_total_loss:
        print(f"epoch{epoch+1} model saved")
        logging.info(f"epoch{epoch+1} saved")
        torch.save(model.state_dict(), f'./T5model_best.hdf5')
        best_loss = val_total_loss


epoch 1 start!


  0%|          | 0/18900 [00:00<?, ?it/s]

  0%|          | 0/4725 [00:00<?, ?it/s]

epoch1 total loss:2.7482, val_total loss:1.9418
epoch1 model saved


## タイトルの予測

In [18]:
# epoch数が最も高いモデルを選択してください。
best_epoch = 1
model_path = './T5model_best.hdf5'
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [19]:
def pred_text(X_test):
    # datasetとdataloaderの作成
    batch_size=1
    test_ds = NarouDataset(input_texts=X_test,input_max_len=256)
    test_dl = torch.utils.data.DataLoader(
        test_ds, batch_size=batch_size, shuffle=False)  
    pred_texts = []
    model.eval()
    for iter_num, (input_ids, input_masks) in tqdm(enumerate(test_dl),total = len(test_dl)):
        optimizer.zero_grad()
        if cuda:
            input_ids, input_masks =\
            input_ids.cuda(), input_masks.cuda()
        with torch.no_grad():
            # forward(順伝搬)
            outputs = model.generate(input_ids)
            for pred_ids in outputs:
                if (pred_ids==1).nonzero().shape[0]:
                    pred_texts.append(tokenizer.decode(pred_ids[1:(pred_ids==1).nonzero()[0,0]]))
                else:
                    pred_texts.append(tokenizer.decode(pred_ids[1:]))
    return pred_texts

In [21]:
# randomで100件抽出
test_index = np.random.choice(len(X_test),100,replace=False)
X_test_sample = X_test.iloc[test_index]
y_test_sample = y_test.iloc[test_index]
preds = pred_text(X_test_sample)

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1301: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [22]:
test_df = pd.DataFrame({
    "title":y_test_sample,    
    "pred_title":preds,
    "story":X_test_sample
})
test_df["len"] = test_df["story"].map(len)

予測結果の表示

In [23]:
test_df[(test_df["len"]>100) & (~test_df["pred_title"].str.contains("<unk>"))]

,title,pred_title,story,len
433,幼女魔王伝 ～幼女にされた元魔王はそれでも最強でした～,幼女魔王の冒険,裏切り者の策略により幼女になる呪いをかけられてしまった魔王ゼノファリウス。人間の世界に行き呪いを解く手がかりを探すも、常識外れな魔力が様々な事件を引き起こしてしまいーー！？\n果たして元の体に戻り魔王の座を取り戻すことができるのか！？\n体は弱くとも魔力は最強！化け物幼女が魔王になるための冒険が今始まる！,151
417,ゲームサウンドとアニソンでマーチング♪,マーチングマーチング,今年はゲームサウンドとアニソンでマーチングします！\n\n中学生になってマーチング部に入部してみた。\n\n同じ楽器を演奏するのは、幼馴染や気の合う子たちばかり。\n\nストレスフリーな、のんびりとした部活動ライフを送るのです。,108
265,カコイマミライ ～時を刻まない島～,時を刻まない島,過去、現在、未来\n\n 時は大きくこの３つに分けることができる。\n だが、とある事件により、時の流れで重要な役割をもつ\n 現在──“今”が止まった。\n\n その事件に巻き込まれることとなったのが、本作の高校生の主人公 “桐島 来夢(ライム)” \n\n ライムは朝学校に向かう途中、謎の黒い空間……\n“時空の歪み”に飲み込まれ、意識を失い“異世界”へと飛ばされてしまう。\n\n 異世界へと飛ばされたはずのライムだが、意識が戻り目を覚ますと……なぜか、見知らぬ島へとたどり着いていた。\n\n この島はとても自然に溢れ、のどかな場所で、一見何てことのない、普通の島のように思えた。...,440
88,虐げられていた姫は、隣国の王子と自国を滅ぼす,腐り切った自国に絶望し、自国を滅ぼそうと考える。,カルテは、とある国の第一王女。しかし彼女は腐り切った自国に絶望し、自らの国を滅ぼそうと考える。だからカルテは隣国の王子グレイに声をかけた。「私と、私の国を滅ぼしませんか？」\n\n王子と婚約するまでのお話。親に罵られるシーンあり。ざまあはそんなないです。逆ハーレム要素あり。,135
500,始まりの女と神の授けた箱,パッチ式電脳空間転送装置,超小型パッチ式電脳空間転送装置、通称ぱっちゃる。その空間には楽しく自由でかつ様々な人とのコミュニケーションが取れる世界があった。\nしかしある日、たった一つの事件が起こった。その事件はこの先に起こるあるひとつの大きな事件へと発展していく。\n主人公である柄木智（からき さとし）はそこで起こった真相を知るため、事件の謎を解くためにその世界へと向かった。その先で待っていた真実とは。,188
205,本好きのティアラは世界一の美少女!?私に平穏な学園生活を下さい!～天才魔術師の二人に守られ、過酷な運命に抗う～,ドラゴンの聖地,本と平穏な日々を過ごしたい少女ティアラ・マルクスは、進学先を実家から遠くの学校を選択し、地味で目立たない様に学園生活を過ごしていた。しかし、地味すぎて誰からもチームの誘いを受けられなかったティアラは、同じく学園に通う天才魔術師の二人にチームに誘われてしまった。お似合いの二人の間に地味な生徒であるティアラが入ってしまい、別の意味で目立つ様になってしまったティアラは、一部の生徒達から日々嫌がらせを受ける様になる。しかし天才魔術師の二人がティアラを守り続ける。ある日、天才魔術師の一人リック・スピルトンに素顔を見られてしまい、告白されてしまう。そしてもう一人の天才魔術師であるピアス・リエライの...,352
92,はみだし者達のカーニバル,グラーヴ、アザルス大陸の交易都市,ここはグラーヴ、アザルス大陸のほぼ中央に位置する比較的大きな交易都市である。\n 一応は王国に属するが、交易都市であることと周囲の帝国、エルフ領、魔国と接する為、様々な人種が行き交う。\n そして、それらの国々から流れてきたクセ者達と、そんな彼等に見劣りしない変人な住民達が住む街である。\n\n これは、【俺が良ければ全て良し】をモットーにしているような彼等の物語。\n\n\n\n\n\n注意、超絶駄文なので何一つ期待をしないでお読みくださいm(_ _)m,221
118,狼少年が少女の為についた嘘。,狼が来たとウソをつき続ける少年と,ある日、突然ウソしか言えなくなってしまった少女がおりました。少女が猫を指差し犬だと言ったり、食事に毒が入っていると言い出したりしたので、両親は少女をウソつき呼ばわりし、家を追い出してしまいました。\n そして少女は出会います。狼が来たとウソをつき続ける少年と…………,133
104,罪人となった俺は魔物になって異世界を救います,俺の人生にない行動をとったせいで世界を狂わしたらしい,平凡な高校生だった俺こと神崎龍馬は歩きながらスマホをいじっていたらトレーラーに吹っ飛ばされてあの世に旅立った。\n そして目が覚めると目の前には自称神様が。\n 俺の人生にない行動をとったせいで何百人もの人生を狂わしたらしい。\n 危うく拷問を受けて魂を砕かれそうになるも滅亡の危機に貧している世界を救うことで許すと言われた俺はその世界に転生するも転生した先は人間に攻めこまれている最中の鬼族だった。\n なんとか窮地を脱するも次々と遅い来る災難に俺は死なずに目的を果たせるのか？\n いつも通り駄文です。一部オリジナル魔物を登場します。,283
401,ポンコツ勇者と幼女魔王の秘密組織育成計画〜最強同士が手を組んで美少女100人育てます〜,咎人の衆,──力が欲しいか？\n\n世界を敵に回し、神さえも降す力を手に入れる為に、人は何をすべきだろうか。\n\nとある秘密組織の幹部となったポンコツ鬼畜勇者と天然サイコパス魔王は、部下を鍛える任を授かっている。\n\nその組織の名は通称【黒の方舟】。数十人の女性メンバーで構成された咎人の衆である。\n\n世界を裏から牛耳り、いつかは天でさえも敵に回さんとするその組織は、笑顔溢れる日常と、おぞましい特訓により成り立っていて──\n\n\n＊コメディ要素強め。\n＊ギャグ一辺倒\n＊渋めの下ネタあり。,235
